## NOTEBOOK: Creating a Postgresql database locally and importing the dataset

Import libraries

In [28]:
import pandas as pd
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

Use ipython-sql to run sql on jupyter notebook

In [1]:
%load_ext sql

### Importing CSV into PostgreSQL
- Use [csvkit](https://csvkit.readthedocs.io/en/1.0.3/) to convert csv into postgresql

In [11]:
!csvsql --db postgresql:///postgres --insert ../data/raw/ehresp_2014.csv

(psycopg2.ProgrammingError) relation "ehresp_2014" already exists
 [SQL: '\nCREATE TABLE ehresp_2014 (\n\ttucaseid DECIMAL NOT NULL, \n\ttulineno BOOLEAN NOT NULL, \n\teeincome1 DECIMAL NOT NULL, \n\terbmi DECIMAL NOT NULL, \n\terhhch DECIMAL NOT NULL, \n\terincome DECIMAL NOT NULL, \n\terspemch DECIMAL NOT NULL, \n\tertpreat DECIMAL NOT NULL, \n\tertseat DECIMAL NOT NULL, \n\tethgt DECIMAL NOT NULL, \n\tetwgt DECIMAL NOT NULL, \n\teudietsoda DECIMAL NOT NULL, \n\teudrink DECIMAL NOT NULL, \n\teueat DECIMAL NOT NULL, \n\teuexercise DECIMAL NOT NULL, \n\teuexfreq DECIMAL NOT NULL, \n\teufastfd DECIMAL NOT NULL, \n\teufastfdfrq DECIMAL NOT NULL, \n\teuffyday DECIMAL NOT NULL, \n\teufdsit DECIMAL NOT NULL, \n\teufinlwgt DECIMAL NOT NULL, \n\teusnap DECIMAL NOT NULL, \n\teugenhth DECIMAL NOT NULL, \n\teugroshp DECIMAL NOT NULL, \n\teuhgt DECIMAL NOT NULL, \n\teuinclvl DECIMAL NOT NULL, \n\teuincome2 DECIMAL NOT NULL, \n\teumeat DECIMAL NOT NULL, \n\teumilk DECIMAL NOT NULL, \n\teuprpmel DE

### Connecting to the database

In [2]:
%sql postgresql://localhost/postgres

'Connected: @postgres'

### Explore Table

In [6]:
%%sql 
SELECT
    * 
FROM ehresp_2014 
    LIMIT 5;

 * postgresql://localhost/postgres
5 rows affected.


tucaseid,tulineno,eeincome1,erbmi,erhhch,erincome,erspemch,ertpreat,ertseat,ethgt,etwgt,eudietsoda,eudrink,eueat,euexercise,euexfreq,eufastfd,eufastfdfrq,euffyday,eufdsit,eufinlwgt,eusnap,eugenhth,eugroshp,euhgt,euinclvl,euincome2,eumeat,eumilk,euprpmel,eusoda,eustores,eustreason,eutherm,euwgt,euwic,exincome1
20140101140007,True,-2,33.200001,1,-1,-1,30,2,0,0,-1,2,1,2,-1,2,-1,-1,1,5202086,1,1,1,60,5,-2,1,2,1,-1,2,1,2,170,1,2
20140101140011,True,1,22.700001,3,1,-1,45,14,0,0,-1,2,1,2,-1,1,1,2,1,29400000,2,2,3,63,5,-1,1,2,1,-1,1,2,2,128,2,0
20140101140028,True,2,49.400002,3,5,-1,60,0,0,0,-1,1,2,2,-1,2,-1,-1,1,26000000,2,5,2,62,5,2,-1,-1,2,2,-1,-1,-1,270,2,12
20140101140063,True,-2,-1,3,-1,-1,0,0,0,-1,2,1,2,2,-1,2,-1,-1,1,2728880,2,2,1,64,5,-2,2,2,1,1,2,6,-1,-2,2,2
20140101140168,True,2,31,3,5,-1,65,0,0,0,-1,1,2,1,5,2,-1,-1,1,17500000,1,4,1,69,5,2,1,2,1,2,1,1,2,210,1,0


How many people have weight>200 lbs?

In [36]:
%%sql
SELECT 
    COUNT(tucaseid)
FROM ehresp_2014
    WHERE euwgt>200;


 * postgresql://localhost/postgres
1 rows affected.


count
2500


How many people have >30 bmi, or OBESE

In [37]:
%%sql
SELECT 
    COUNT(tucaseid)
FROM ehresp_2014
    WHERE erbmi > 30;


 * postgresql://localhost/postgres
1 rows affected.


count
3040


How many people have <30 bmi, or NOT OBESE

In [38]:
%%sql
SELECT 
    COUNT(tucaseid)
FROM ehresp_2014
    WHERE erbmi < 30;


 * postgresql://localhost/postgres
1 rows affected.


count
8084


Any negative answers? 
- blanks (-1)
- don't know (-2)
- refused to answer (-3_

How many people don't exercise and are obese?

In [42]:
%%sql
SELECT 
    COUNT(tucaseid)
FROM ehresp_2014
    WHERE (euexfreq<0 AND erbmi>30);

 * postgresql://localhost/postgres
1 rows affected.


count
1390


How many people exercise and yet still obese?

In [41]:
%%sql
SELECT
    COUNT(tucaseid)
FROM ehresp_2014
    WHERE (euexfreq>0 AND erbmi>30);

 * postgresql://localhost/postgres
1 rows affected.


count
1650


Locate people who order takeouts/order fastfood and obese, ordered by weight 

In [44]:
%%sql
SELECT 
    *
FROM ehresp_2014
    WHERE (erbmi>30 AND eufastfdfrq>0)
        ORDER BY
            euwgt DESC
    LIMIT 8;

 * postgresql://localhost/postgres
8 rows affected.


tucaseid,tulineno,eeincome1,erbmi,erhhch,erincome,erspemch,ertpreat,ertseat,ethgt,etwgt,eudietsoda,eudrink,eueat,euexercise,euexfreq,eufastfd,eufastfdfrq,euffyday,eufdsit,eufinlwgt,eusnap,eugenhth,eugroshp,euhgt,euinclvl,euincome2,eumeat,eumilk,euprpmel,eusoda,eustores,eustreason,eutherm,euwgt,euwic,exincome1
20140403140670,True,2,43.599998,1,3,5,60,30,0,1,2,1,1,2,-1,1,15,1,1,4300922,2,3,2,74,5,1,-1,-1,2,1,-1,-1,-1,340,1,0
20140403140731,True,1,53.200001,3,1,-1,45,15,0,1,-1,1,1,2,-1,1,3,2,1,3230588,1,3,1,67,5,-1,1,2,1,2,3,1,2,340,2,0
20140302142086,True,2,46.099998,3,5,-1,30,0,0,0,-1,2,2,2,-1,1,1,2,1,2508998,2,4,1,72,5,2,1,2,1,-1,1,6,2,340,-1,0
20140301141057,True,1,73.599998,3,1,5,55,0,0,1,-1,2,2,1,4,1,1,2,1,5869593,2,2,1,57,5,-1,1,2,1,-1,1,1,2,340,1,0
20140302141278,True,2,47.400002,3,3,5,45,10,0,1,-1,2,1,2,-1,1,3,1,1,13500000,2,3,2,71,5,1,1,2,1,-1,-1,-1,2,340,2,0
20140112130956,True,1,42.5,3,1,5,50,-2,0,0,2,1,-2,1,4,1,14,1,1,4661715,2,3,2,75,5,-1,-1,-1,2,1,-1,-1,-1,340,2,0
20140302140854,True,1,43.599998,3,1,5,120,0,0,1,-1,1,2,2,-1,1,3,2,1,8119375,2,3,2,74,5,-1,-1,-1,2,2,-1,-1,-1,340,2,0
20140403141911,True,2,51.700001,3,5,-1,60,2,0,1,-1,2,1,1,1,1,2,2,2,5197074,1,4,1,68,5,2,1,2,1,-1,2,1,2,340,2,0


How many people eat/serve meat and use meat thermometer

In [48]:
%%sql
SELECT 
    COUNT(tucaseid)
FROM ehresp_2014
    WHERE (eumeat=1 AND eutherm=1);

 * postgresql://localhost/postgres
1 rows affected.


count
846


Look for people who are under 100 lbs

In [46]:
%%sql
SELECT 
    *  
FROM ehresp_2014
    WHERE (euwgt<100)
        LIMIT 4;

 * postgresql://localhost/postgres
4 rows affected.


tucaseid,tulineno,eeincome1,erbmi,erhhch,erincome,erspemch,ertpreat,ertseat,ethgt,etwgt,eudietsoda,eudrink,eueat,euexercise,euexfreq,eufastfd,eufastfdfrq,euffyday,eufdsit,eufinlwgt,eusnap,eugenhth,eugroshp,euhgt,euinclvl,euincome2,eumeat,eumilk,euprpmel,eusoda,eustores,eustreason,eutherm,euwgt,euwic,exincome1
20140101140063,True,-2,-1,3,-1,-1,0,0,0,-1,2,1,2,2,-1,2,-1,-1,1,2728880,2,2,1,64,5,-2,2,2,1,1,2,6,-1,-2,2,2
20140101140872,True,1,-1,3,1,5,0,10,0,-1,-1,1,1,1,4,2,-1,-1,1,8319956,2,4,3,65,5,-1,1,2,1,2,1,2,1,-3,2,0
20140101141672,True,2,-1,3,5,5,25,15,0,-1,1,1,1,2,-1,1,2,2,1,28200000,1,2,1,64,5,2,1,2,1,1,2,1,1,-5,1,0
20140101141837,True,-2,-1,3,-1,-1,150,0,-1,-1,-1,2,2,1,3,1,3,2,1,12400000,2,3,1,-2,5,-2,1,2,1,-1,1,3,2,-2,-1,2


Now, add another columns corresponding to obesity classification
- 0 for Not Obese
- 1 for Obese

### Select columns of interest and create a view table

In [8]:
%%sql 
SELECT 
    ertpreat, ertseat, euexfreq, eufastfdfrq, eugenhth, erbmi 
FROM ehresp_2014 
    LIMIT 5;

 * postgresql://localhost/postgres
5 rows affected.


ertpreat,ertseat,euexfreq,eufastfdfrq,eugenhth,erbmi
30,2,-1,-1,1,33.200001
45,14,-1,1,2,22.700001
60,0,-1,-1,5,49.400002
0,0,-1,-1,2,-1
65,0,5,-1,4,31


Create a VIEW table with columns of interest AND add another column based on conditionals 

In [15]:
%%sql
CREATE VIEW main_table AS (
    SELECT ertpreat, ertseat, euexfreq, eufastfdfrq, eugenhth, erbmi,
        CASE WHEN erbmi>30 THEN 'NotObese'
                   ELSE 'Obese'
        END
    FROM ehresp_2014
        -- NO LIMIT
        );

 * postgresql://localhost/postgres
Done.


[]

Check out the VIEW table

In [16]:
%%sql
SELECT *
FROM main_table
    LIMIT 5;

 * postgresql://localhost/postgres
5 rows affected.


ertpreat,ertseat,euexfreq,eufastfdfrq,eugenhth,erbmi,case
30,2,-1,-1,1,33.200001,NotObese
45,14,-1,1,2,22.700001,Obese
60,0,-1,-1,5,49.400002,NotObese
0,0,-1,-1,2,-1,Obese
65,0,5,-1,4,31,NotObese


Renaming VIEW table column names, so that they are more intuitive

In [17]:
%%sql
ALTER TABLE main_table
    RENAME COLUMN ertpreat 
        TO timeEat;
        
ALTER TABLE main_table
    RENAME COLUMN ertseat
        TO timesnack;

ALTER TABLE main_table
    RENAME COLUMN euexfreq
        TO exercisefreq;

ALTER TABLE main_table
    RENAME COLUMN eufastfdfrq
        TO fastfdfreq;
    
ALTER TABLE main_table
    RENAME COLUMN eugenhth
        TO genhealth;
        
ALTER TABLE main_table 
    RENAME COLUMN erbmi
        TO bmi;
        
ALTER TABLE main_table
    RENAME COLUMN case
        TO obese;

 * postgresql://localhost/postgres
Done.
Done.
Done.
Done.
Done.
Done.
(psycopg2.ProgrammingError) syntax error at or near "case"
LINE 2:     RENAME COLUMN case
                          ^
 [SQL: 'ALTER TABLE main_table\n    RENAME COLUMN case\n        TO obese;'] (Background on this error at: http://sqlalche.me/e/f405)


Check the renamed column names in the main_table

In [37]:
%%sql 
SELECT * 
    FROM main_table
        LIMIT 5;

   postgresql://localhost/
 * postgresql://localhost/postgres
5 rows affected.


timeeat,timesnack,exercisefreq,fastfdfreq,genhealth,bmi
30,2,-1,-1,1,33.200001
45,14,-1,1,2,22.700001
60,0,-1,-1,5,49.400002
0,0,-1,-1,2,-1
65,0,5,-1,4,31


### Extract VIEW table into pandas dataframe

In [38]:
# local machine
params = {
    'host': '127.0.0.1',
    'port': 5432
}

In [39]:
# Connect to database
connection = connect(**params,dbname='postgres')
cursor = connection.cursor()

In [44]:
# move cursor and fetch table
cursor.execute('SELECT * FROM main_table;')
df = pd.DataFrame(data = cursor.fetchall(),
                  columns = ['timeeat', 'timesnack','exercisefreq',
                            'fastfdfreq','genhealth','bmi'])

In [45]:
df.head()

,timeeat,timesnack,exercisefreq,fastfdfreq,genhealth,bmi
0,30,2,-1,-1,1,33.200001
1,45,14,-1,1,2,22.700001
2,60,0,-1,-1,5,49.400002
3,0,0,-1,-1,2,-1
4,65,0,5,-1,4,31


- once it's in pandas dataframe, cleaning and modeling can be done as usual (see notebooks for Step2 and Step3)

---